In [1]:
import sklearn
from hmmlearn import hmm
import numpy as np
import requests
import json
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.preprocessing.sequence import pad_sequences


C:\Users\smahe\AppData\Local\Programs\Python\Python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Retrieve Card Data#

Pull the card text of all playable Hearthstone cards from https://hearthstonejson.com/

In [148]:
response = requests.get("https://api.hearthstonejson.com/v1/25252/enUS/cards.collectible.json")

print(response.status_code)


200


In [152]:
cards_list = response.json()

cards_w_text = list(filter(lambda x:'text'in x.keys(), cards_list))

card_desc = list(map(lambda x:  x['text'], cards_w_text))

print(card_desc[:5])

['Deal $8 damage to a minion.', '<b>Secret:</b> When a friendly minion dies, summon a random minion with the same Cost.', 'Your Hero Power deals 1 extra damage.', 'Deal $2 damage to a minion. This spell gets double bonus from <b>Spell Damage</b>.', 'Transform a minion into a 4/2 Boar with <b>Charge</b>.']


# Clean the text#

Remove punctuation and strange characters

In [153]:
lines = (filter(lambda x: x != 'NIL', card_desc))

lines = (map(lambda x: x.replace('\n', ' ').replace('\xa0',' ').replace('[x]', '').replace('{1}','')
                 .replace('{0}', '').replace('@', '').replace(';', ''), lines))

lines = (map(lambda x: x.replace(".", "").replace("!", "").replace("?", "").replace("<b>", "").replace("</b>", "")
             .replace("</i>", "").replace("<i>", ""), lines))

lines = (map(lambda x: x.replace("$", "").replace("+", "")
             .replace("(", "").replace(")", "").replace("#", "").replace(":", ""), lines))

lines = list(map(lambda x: x.lower(), lines))

print(lines[:5])


['deal 8 damage to a minion', 'secret when a friendly minion dies, summon a random minion with the same cost', 'your hero power deals 1 extra damage', 'deal 2 damage to a minion this spell gets double bonus from spell damage', 'transform a minion into a 4/2 boar with charge']


# Create a words mapping #

Create a map of all words in the card descriptions and encode them.

In [154]:
flat_list = [item for sublist in lines for item in sublist]

chars = sorted(list(set(flat_list)))

mapping = dict((c, i) for i, c in enumerate(chars))

words = list(map(lambda x: x.split(" "), lines))

flat_list_words = [item for sublist in words for item in sublist]

all_words = sorted(list(set(flat_list_words)))
word_map = dict((c, i) for i, c in enumerate(all_words))

print(np.mean(np.array(list(map(lambda x: len(x), words)))))

8.873158231902627


In [155]:
sequences = list()
for line in lines:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    sequences.append((np.array(encoded_seq)))

# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 44


In [156]:
temp = []
lengths_arr = []
for line in words:
    # integer encode line
    encoded_seq = ([np.array(word_map[word]) for word in line])
    lengths_arr.append(len(encoded_seq))
    # store
    temp += encoded_seq

# vocabulary size
vocab_size = len(mapping)
print(vocab_size)

44


# Train a Hidden Markov Model#

The Hidden Markov Model learns the hidden states composing a Hearthstone card description, <br />
and builds a random sample that should resemble some card description.

In [158]:
sequences = np.array(temp)
sequences = (np.array(sequences.reshape(-1,1)))

hmm_model = hmm.MultinomialHMM(n_components=6)
hmm_model.fit(sequences, lengths_arr)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=6,
        n_iter=10, params='ste',
        random_state=<mtrand.RandomState object at 0x0000022FCF1656C0>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=False)

# Generate sequence from HMM#
The sequence should generate something resembling a card description. <br />
However, perhaps due to sample size, results are underwhelming.

In [159]:
predictions = hmm_model.sample(20)

inv_map = {v: k for k, v in word_map.items()}

output = ""

for char_arr in predictions[0]:
    
    output += inv_map[char_arr[0]] + " "

print(output)

deathrattle your enemy  silver each cost 50% player's one 1 taunt random your a damage this give health hand 


# Create new sequences#
Break down card text into sequences of a set length. 

In [81]:
joined_lines  = " ".join(lines)

length = 10
sequences = list()
for line in lines:
    for i in range(length, len(line)):
        # select sequence of tokens
        seq = line[i-length:i+1]
        # store
        sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

new_input = sequences

Total Sequences: 59309


In [125]:
print(new_input[:10])

['deal 8 dama', 'eal 8 damag', 'al 8 damage', 'l 8 damage ', ' 8 damage t', '8 damage to', ' damage to ', 'damage to a', 'amage to a ', 'mage to a m']


# Create a character mapping #

Create a map of all characters in the card descriptions and encode them.

In [160]:
chars = sorted(list(set(joined_lines)))
mapping = dict((c, i) for i, c in enumerate(chars))

vocab_size = len(mapping)

sequences = list()
for line in new_input:
	# integer encode line
	encoded_seq = [mapping[char] for char in line]
	# store
	sequences.append(np.array(encoded_seq))

In [161]:
sequences = np.array(sequences)

np.random.shuffle(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

In [162]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]

X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)

# Build an LSTM model#

In [100]:
# define model
n_batch = len(X)
model = Sequential()
model.add(LSTM(128, input_shape=( X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 10, 128)           89088     
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 45)                5805      
Total params: 226,477
Trainable params: 226,477
Non-trainable params: 0
_________________________________________________________________
None


# Train the RNN#

In [ ]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=2)

# Generate sequence from the RNN#

In [164]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text

    for _dummy in range(n_chars):
        encoded = [mapping[char] for char in in_text]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')        
        encoded = to_categorical(encoded, num_classes=len(mapping))
    
        encoded = encoded.reshape(1, encoded.shape[1], encoded.shape[2])
        
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
                
        # append to input
        in_text += char
    return in_text

# test start of rhyme
#print(generate_seq(model, mapping, 10, 'battlecry:', 20))
# test mid-linery
text = "if your deck"
print(generate_seq(model, mapping, 10, text, 100))
# test not in original
#print(generate_seq(model, mapping, 10, 'hello worl', 20))

if your deck has no duplicates, summon a random minion in your hand it costs 2 less for each minion that died th


#Interesting output#

In [ ]:
battlecry destroy a murloc and gain 2/2 for each card in your hand

give a minion 2 attack this turn and 4 armor

divine a random 4-cost minion for your opponent casts a spell

costs 1 less for each minion that died this game, summon thaddius

deal 2 damage to all enemies restore 2 health to your hero and gain 2 attack this turn

choose one - give your other minions have 1 attack and health

summon a 2/2 ghoul into your opponent's deck

battlecry destroy a minion in your hand it costs 2 less for each minion that died this game

echo taunt and divine shield

windfury while damaged

if your deck has no duplicates, summon a random minion in your hand